In [1]:
import os
import torch
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.model_zoo import get_config_file

INPUT_DIR = "/home/fahadabul/mask_rcnn_skyhub/test/final"
MODEL_PATH_WRINKLE = "/home/fahadabul/mask_rcnn_skyhub/latest_image_mask_rcnn_wrinkle/best_model_wrinkle.pth"
OUTPUT_DIR = "/home/fahadabul/mask_rcnn_skyhub/test/model_eval"
COCO_GT_JSON = "/home/fahadabul/mask_rcnn_skyhub/test/test.json"
DATASET_NAME = "wrinkle_eval"

if "torn_wrinkle_dataset" not in MetadataCatalog.list():
    MetadataCatalog.get("torn_wrinkle_dataset").set(thing_classes=["torn", "wrinkle"])
metadata = MetadataCatalog.get("torn_wrinkle_dataset")

def load_model(model_path, class_names):
    cfg = get_cfg()
    cfg.merge_from_file(get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(class_names)
    cfg.MODEL.WEIGHTS = model_path
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3
    cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    return DefaultPredictor(cfg)

predictor_wrinkle = load_model(MODEL_PATH_WRINKLE, ["wrinkle"])

if DATASET_NAME not in DatasetCatalog.list():
    register_coco_instances(DATASET_NAME, {}, COCO_GT_JSON, INPUT_DIR)

from detectron2.data import build_detection_test_loader
val_loader = build_detection_test_loader(predictor_wrinkle.cfg, DATASET_NAME)

evaluator = COCOEvaluator(DATASET_NAME, output_dir=OUTPUT_DIR)
results = inference_on_dataset(predictor_wrinkle.model, val_loader, evaluator)

def print_coco_metrics(results, label=""):
    for key in ["bbox", "segm"]:
        if key in results:
            print(f"\n{label} {key.upper()} metrics:")
            print(f"  mAP (IoU=0.50:0.95): {results[key].get('AP', 'N/A'):.3f}")
            print(f"  mAP50 (IoU=0.50):    {results[key].get('AP50', 'N/A'):.3f}")
            print(f"  mAP75 (IoU=0.75):    {results[key].get('AP75', 'N/A'):.3f}")
            print(f"  AP_small:            {results[key].get('APs', 'N/A')}")
            print(f"  AP_medium:           {results[key].get('APm', 'N/A')}")
            print(f"  AP_large:            {results[key].get('APl', 'N/A')}")

print_coco_metrics(results)



/home/fahadabul/mask_rcnn_skyhub/skyenv/lib/python3.10/site-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.031
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.092
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.014
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.058
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.011
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.050
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.058
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:

In [6]:
import os
import torch
import json
from tqdm import tqdm

from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.model_zoo import get_config_file

from sahi.models.detectron2 import Detectron2DetectionModel
from sahi.predict import get_sliced_prediction

# --- PATHS ---
INPUT_DIR = "/home/fahadabul/mask_rcnn_skyhub/test/final"
MODEL_PATH_WRINKLE = "/home/fahadabul/mask_rcnn_skyhub/latest_image_mask_rcnn_wrinkle/best_model_wrinkle.pth"
OUTPUT_DIR = "/home/fahadabul/mask_rcnn_skyhub/test/model_eval"
COCO_GT_JSON = "/home/fahadabul/mask_rcnn_skyhub/test/test.json"
DATASET_NAME = "wrinkle_eval"

# --- REGISTER DATASET ---
if "torn_wrinkle_dataset" not in MetadataCatalog.list():
    MetadataCatalog.get("torn_wrinkle_dataset").set(thing_classes=["torn", "wrinkle"])
metadata = MetadataCatalog.get("torn_wrinkle_dataset")

# --- LOAD MODEL FUNCTION ---
def load_model(model_path, class_names):
    cfg = get_cfg()
    cfg.merge_from_file(get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(class_names)
    cfg.MODEL.WEIGHTS = model_path
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3
    cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    return DefaultPredictor(cfg)

# --- LOAD MODEL ---
predictor_wrinkle = load_model(MODEL_PATH_WRINKLE, ["wrinkle"])

# --- REGISTER DATASET IF NOT ALREADY ---
if DATASET_NAME not in DatasetCatalog.list():
    register_coco_instances(DATASET_NAME, {}, COCO_GT_JSON, INPUT_DIR)

# --- WRAP WITH SAHI ---
sahi_model = Detectron2DetectionModel(
    model_path=MODEL_PATH_WRINKLE,  # your .pth path
    config_path=get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"),  # config file path
    device="cuda" if torch.cuda.is_available() else "cpu",
    confidence_threshold=0.3,
)

# --- PERFORM SAHI INFERENCE ON ALL TEST IMAGES ---
coco_predictions = []

dataset_dicts = DatasetCatalog.get(DATASET_NAME)

for data in tqdm(dataset_dicts, desc="Running SAHI slicing inference"):
    image_path = data["file_name"]
    image_id = data["image_id"]

    result = get_sliced_prediction(
        image=image_path,
        detection_model=sahi_model,
        slice_height=512,
        slice_width=512,
        overlap_height_ratio=0.2,
        overlap_width_ratio=0.2,
    )

    for pred in result.object_prediction_list:
        bbox = pred.bbox.to_xywh()
        coco_predictions.append({
            "image_id": image_id,
            "category_id": pred.category.id,
            "bbox": [bbox.minx, bbox.miny, bbox.width, bbox.height],
            "score": pred.score.value,
        })

# --- SAVE PREDICTIONS ---
os.makedirs(OUTPUT_DIR, exist_ok=True)
predictions_path = os.path.join(OUTPUT_DIR, "sahi_coco_predictions.json")
with open(predictions_path, "w") as f:
    json.dump(coco_predictions, f)

print(f"\n✅ Saved SAHI predictions to {predictions_path}")

# --- EVALUATE USING COCOEvaluator ---
from detectron2.evaluation import COCOEvaluator

evaluator = COCOEvaluator(DATASET_NAME, output_dir=OUTPUT_DIR)

# To load external predictions, use evaluator.load_json_results
# Normally detectron2 expects model inference inside inference_on_dataset()
# Since we already have predictions, we trick it using the evaluator directly

evaluator._predictions = coco_predictions
results = evaluator.evaluate()

# --- PRINT RESULTS ---
def print_coco_metrics(results, label=""):
    for key in ["bbox", "segm"]:
        if key in results:
            print(f"\n{label} {key.upper()} metrics:")
            print(f"  mAP (IoU=0.50:0.95): {results[key].get('AP', 'N/A'):.3f}")
            print(f"  mAP50 (IoU=0.50):    {results[key].get('AP50', 'N/A'):.3f}")
            print(f"  mAP75 (IoU=0.75):    {results[key].get('AP75', 'N/A'):.3f}")
            print(f"  AP_small:            {results[key].get('APs', 'N/A')}")
            print(f"  AP_medium:           {results[key].get('APm', 'N/A')}")
            print(f"  AP_large:            {results[key].get('APl', 'N/A')}")

print_coco_metrics(results)


05/08/2025 18:37:55 - INFO - detectron2.checkpoint.detection_checkpoint -   [DetectionCheckpointer] Loading from /home/fahadabul/mask_rcnn_skyhub/latest_image_mask_rcnn_wrinkle/best_model_wrinkle.pth ...
05/08/2025 18:37:55 - INFO - fvcore.common.checkpoint -   [Checkpointer] Loading from /home/fahadabul/mask_rcnn_skyhub/latest_image_mask_rcnn_wrinkle/best_model_wrinkle.pth ...


/home/fahadabul/mask_rcnn_skyhub/skyenv/lib/python3.10/site-packages/detectron2/model_zoo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml not available in Model Zoo!


05/08/2025 18:37:55 - INFO - detectron2.checkpoint.detection_checkpoint -   [DetectionCheckpointer] Loading from /home/fahadabul/mask_rcnn_skyhub/latest_image_mask_rcnn_wrinkle/best_model_wrinkle.pth ...
05/08/2025 18:37:55 - INFO - fvcore.common.checkpoint -   [Checkpointer] Loading from /home/fahadabul/mask_rcnn_skyhub/latest_image_mask_rcnn_wrinkle/best_model_wrinkle.pth ...
05/08/2025 18:37:56 - WARNING - fvcore.common.checkpoint -   Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (2, 1024) in the checkpoint but (81, 1024) in the model! You might want to double check if this is expected.
05/08/2025 18:37:56 - WARNING - fvcore.common.checkpoint -   Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (2,) in the checkpoint but (81,) in the model! You might want to double check if this is expected.
05/08/2025 18:37:56 - WARNING - fvcore.common.checkpoint -   Skip loading par

Performing prediction on 80 slices.


Running SAHI slicing inference:   5%|▍         | 1/21 [02:35<51:39, 154.99s/it]

...

Performing prediction on 80 slices.


Running SAHI slicing inference:  95%|█████████▌| 20/21 [57:12<02:28, 148.37s/it]

Performing prediction on 80 slices.


Running SAHI slicing inference: 100%|██████████| 21/21 [59:50<00:00, 170.96s/it]
05/08/2025 19:37:46 - WARNING - detectron2.evaluation.coco_evaluation -   [COCOEvaluator] Did not receive valid predictions.



✅ Saved SAHI predictions to /home/fahadabul/mask_rcnn_skyhub/test/model_eval/sahi_coco_predictions.json


In [ ]:
import os
import cv2
import torch
import numpy as np
from ultralytics import YOLO  # YOLOv8
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.model_zoo import get_config_file
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import Instances
# Paths
INPUT_DIR = "/home/amruta/Downloads/sky/training_data/set2_input"
OUTPUT_DIR = "/home/amruta/Downloads/sky/training_data/output_set2_v3_model/"
MODEL_PATH_WRINKLE = "/home/amruta/Downloads/sky/models/v3_maskrcnn/output/model_final.pth"

# Register metadata
if "torn_wrinkle_dataset" not in MetadataCatalog.list():
    MetadataCatalog.get("torn_wrinkle_dataset").set(thing_classes=["torn", "wrinkle"])
metadata = MetadataCatalog.get("torn_wrinkle_dataset")



# Load Mask R-CNN model
def load_model(model_path, class_names):
    cfg = get_cfg()
    cfg.merge_from_file(get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(class_names)
    cfg.MODEL.WEIGHTS = model_path
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3
    cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    return DefaultPredictor(cfg)


predictor_wrinkle = load_model(MODEL_PATH_WRINKLE, ["wrinkle"])

def relabel_instances(instances, class_offset):
    if len(instances) > 0:
        instances.pred_classes += class_offset
    return instances

def offset_instances(instances, offset_x, offset_y, full_image_shape):
    instances = instances.to("cpu")
    instances.pred_boxes.tensor += torch.tensor([offset_x, offset_y, offset_x, offset_y])
    masks = instances.pred_masks.numpy()
    h, w = masks.shape[1:]
    padded_masks = np.zeros((len(masks), full_image_shape[0], full_image_shape[1]), dtype=np.uint8)
    for i, mask in enumerate(masks):
        padded_masks[i, offset_y:offset_y+h, offset_x:offset_x+w] = mask
    instances.pred_masks = torch.from_numpy(padded_masks)

    # ✅ Set full image size
    instances._image_size = full_image_shape[:2]
    return instances



def process_segmented_images(segmented_dir, output_dir_segmented):
    for root, _, files in os.walk(segmented_dir):
        for file in files:
            if not file.lower().endswith((".jpg", ".jpeg", ".png")):
                continue

            image_path = os.path.join(root, file)
            relative_path = os.path.relpath(root, segmented_dir)
            save_dir = os.path.join(output_dir_segmented, relative_path)
            os.makedirs(save_dir, exist_ok=True)
            output_path = os.path.join(save_dir, file)

            image = cv2.imread(image_path)
            if image is None:
                print(f"Skipping {image_path}: Unable to read image.")
                continue

            height, width = image.shape[:2]

            wrinkle_outputs = predictor_wrinkle(image)
            # outputs = predictor(image)
            all_instances = []

            # Wrinkle class (1)
            offset_wrinkle = offset_instances(wrinkle_outputs["instances"], 0, 0, image.shape[:2])
            offset_wrinkle = relabel_instances(offset_wrinkle, 1)

            if len(offset_wrinkle) > 0:
                all_instances.append(offset_wrinkle)

            if all_instances:
                combined_instances = Instances.cat(all_instances)
                v = Visualizer(image[:, :, ::-1], metadata=metadata, scale=1.0)
                output_image = v.draw_instance_predictions(combined_instances).get_image()[:, :, ::-1]
            else:
                output_image = image

process_segmented_images(INPUT_DIR, OUTPUT_DIR)


